# Chains and LCEL

## 1. Setup

In [1]:
# Import required modules
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv()

# Model name
MODEL_NAME = "gpt-5-mini"


## 2. A Basic Chain

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You respond like a {animal} and add {voice} at the end of your sentences."),
    ("human", "What are the key points for doing {question}?"),
])

# Create the model
model = ChatOpenAI(model=MODEL_NAME)

# Output as a string
str_output_parser = StrOutputParser()

# Create a chain (each component is a Runnable)
my_chain = prompt | model | str_output_parser

# Run the chain
response = my_chain.invoke({"animal": "dog", "voice": "Woof!", "question": "learning English"})

# Display the result
print(response)


Hi friend — I’m a helpful dog ready to give tips for learning English, Woof!  

- Set clear, realistic goals (e.g., “learn 20 words a week” or “hold a 5-minute conversation”) and study a little every day, Woof!  
- Practice the four skills: listening, speaking, reading, and writing, Woof!  
- Prefer meaningful input: watch shows, listen to podcasts, and read about things you enjoy so learning feels like play, Woof!  
- Speak as much as possible, even if you make mistakes; use language exchanges, tutors, or talk to yourself while wagging, Woof!  
- Learn vocabulary in context and use spaced repetition (SRS) to remember words, Woof!  
- Focus on high-frequency words and common grammar patterns before rarer ones, Woof!  
- Work on pronunciation with imitation and shadowing — copy native speakers’ rhythm and sounds, Woof!  
- Produce language actively (write, speak, summarize) rather than just passive study, Woof!  
- Get regular feedback and correct errors so you don’t fossilize mistakes,

## 3. Adding Arbitrary Functions to a Chain

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain

# A function that returns the given string in reverse order.
# The @chain decorator turns it into a RunnableLambda.
@chain
def reverse_string(message):
    return message[::-1]  # Reverse the string

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}\n\nPlease answer the above question concisely."),
])

# Output as a string
str_output_parser = StrOutputParser()

# Create a chain.
# Instead of @chain, you could also wrap it as RunnableLambda(reverse_string) here.
# If it's part of a chain, it may be auto-converted as well.
my_chain = prompt | model | str_output_parser | reverse_string

# Run the chain
response = my_chain.invoke({"question": "Hello!"})

# Display the result
print(response)


.ylesicnoc dnopser ll'I dna ,derewsna tnaw uoy tahw em llet ro noitseuq eht etsap esaelP .evoba noitseuq a ees t'nod I — iH


## 4. Running Multiple Chains in Parallel

In [4]:
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

# Create prompt templates
positive_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an optimist. You always give a positive answer to the user's question."),
    ("human", "{question}"),
])
negative_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a pessimist. You always give a negative answer to the user's question."),
    ("human", "{question}"),
])

# Create the model
model = ChatOpenAI(model=MODEL_NAME)

# Create chains
positive_chain = positive_prompt | model | str_output_parser
negative_chain = negative_prompt | model | str_output_parser

# Create a parallel chain
parallel_chain = RunnableParallel({
    "positive": positive_chain,
    "negative": negative_chain
})

# Run the chain
response = parallel_chain.invoke({"question": "What impact will the evolution of AI have on humans?"})

# Display the result
print(json.dumps(response, ensure_ascii=False, indent=2))


{
  "positive": "Short answer: very large and mostly positive if we shape it well. AI’s evolution will amplify human capabilities, boost prosperity, and open new possibilities in health, learning, creativity, and problem‑solving — while also creating challenges we can manage through good policy, design, and human choices.\n\nKey areas of impact\n\n- Everyday life and productivity\n  - AI will automate routine tasks, so people can focus on higher‑value, creative, and interpersonal work.\n  - Personal assistants and smart tools will save time, reduce friction, and help people make better decisions (scheduling, finances, travel, home management).\n\n- Work and the economy\n  - Many jobs will change rather than disappear; new roles will be created around AI oversight, strategy, data curation, and human-centered services.\n  - Productivity gains can raise wages, lower costs of goods/services, and create new industries and startups.\n  - Reskilling and learning will become central — but soci

In [5]:
# Create a prompt template to summarize the opinions
opinion_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an egalitarian. You summarize the two opinions in a balanced way. Output only the summary."),
    ("human", "Optimistic opinion: {positive}\nPessimistic opinion: {negative}"),
])

opinion_chain = parallel_chain | opinion_prompt | model | str_output_parser

# Run the chain
response = opinion_chain.invoke({"question": "What impact will the evolution of AI have on humans?"})

# Display the result
print(response)


One view sees AI as an overwhelmingly positive, transformative force: it can boost productivity and create new kinds of work, speed medical and scientific advances, personalize education, improve daily life and accessibility, and help tackle environmental and complex global problems. Its main risks — job disruption, biased systems, misuse, concentration of power, and privacy and trust issues — are framed as solvable through retraining and social safety nets, standards and audits, safety research and international cooperation, competition policy and public investment, and clear regulation; practical steps include AI literacy and lifelong learning for individuals, reskilling and human‑centered design by employers and educators, forward‑looking regulation and funding by governments, responsible practices by researchers, and accountability from civil society.  

The opposing view emphasizes likely harms: large‑scale and lasting job loss and wage pressure, greater concentration of wealth an

## 5. Passing Input Through Unchanged

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough

# Create the prompt template
improve_prompt = ChatPromptTemplate.from_messages([
    ("system", "Analyze the user's question, define constraints, and clarify the objective. If necessary, use Markdown notation and break the task down into smaller steps to craft a prompt that yields the best possible output. Output only the final prompt. Do not answer the user's question."),
    ("human", "Question: {prompt}"),
])

# Create the model
model = ChatOpenAI(model=MODEL_NAME)

# Create the prompt-improvement chain
prompt_chain = improve_prompt | model | str_output_parser

# Create the parallel chain
parallel_chain = RunnableParallel({
    "original": RunnablePassthrough(),
    "improve": prompt_chain
})

# Run the chain
response = parallel_chain.invoke({"prompt": "What impact will the evolution of AI have on humans?"})

# Display the result
print(json.dumps(response, ensure_ascii=False, indent=2))

{
  "original": {
    "prompt": "What impact will the evolution of AI have on humans?"
  },
  "improve": "You are an expert analyst with deep knowledge of artificial intelligence (research, economics, social impacts, policy, and risk). Answer the user question: \"What impact will the evolution of AI have on humans?\" Produce a structured, balanced, evidence-informed, and forward-looking analysis suitable for a general audience that also helps policymakers and informed individuals.\n\nConstraints and required structure\n- Begin with a concise executive summary (3–5 brief sentences) capturing the main conclusions and overall tone (balanced, pragmatic).\n- Cover three time horizons separately:\n  - Near-term: 0–5 years\n  - Medium-term: 5–20 years\n  - Long-term: 20+ years\n- For each time horizon, discuss impacts across these domains:\n  - Economy & labor (jobs, productivity, wages)\n  - Inequality & distributional effects\n  - Education & skills\n  - Healthcare & wellbeing\n  - Governan

## 6. Using Tools

In [7]:
from langchain_tavily import TavilySearch

# Initialize the tool
tavily_search = TavilySearch(
    max_results=2,                 # Maximum number of search results to retrieve
    search_depth="basic",          # "basic" (fast) or "advanced" (higher quality)
    include_answer=False,          # Do not include Tavily's short generated answer
    include_raw_content=False,     # Whether to include raw HTML content (note: higher token usage)
    include_images=False,          # Whether to include image URLs
    # include_domains=["go.jp"],   # Restrict search to specific domains
    # exclude_domains=["wikipedia.org"] # Exclude specific domains
)

# A helper function to format only Tavily search results into an LLM-friendly string
def format_tavily_results(tavily_response: dict) -> str:
    results = tavily_response.get("results", [])
    if not results:
        return "(No search results)"

    lines = []
    for i, r in enumerate(results, 1):
        title = r.get("title", "")
        content = r.get("content", "")
        url = r.get("url", "")
        lines.append(f"[{i}] {title}\n{content}\nsource: {url}")
    return "\n\n".join(lines)


In [8]:
# Test the Tavily search tool
response = tavily_search.invoke("What are some well-known souvenirs from Nara Prefecture?")
print(json.dumps(response, ensure_ascii=False, indent=2))

# Separator
print("-" * 20)

# Check the formatted results
print(format_tavily_results(response))


{
  "query": "What are some well-known souvenirs from Nara Prefecture?",
  "follow_up_questions": null,
  "answer": null,
  "images": [],
  "results": [
    {
      "url": "https://spejapa-excursion.com/blog/b-70.html",
      "title": "Famous souvenirs in Nara prefecture - Spejapa excursion",
      "content": "Famous souvenirs in Nara prefecture · 1. Nara Deer Cookies (Shika Senbei) · 2. Nara Uchiwa Fans · 3. Yamato Cha (Nara Green Tea) · 4. Kakinoha Sushi · 5. Nara",
      "score": 0.9328032,
      "raw_content": null
    },
    {
      "url": "https://livejapan.com/en/in-kansai/in-pref-nara/in-nara_ikoma_tenri/article-a2000428/",
      "title": "Love Nara? Take Back Some Great Nara Souvenirs From These 3 ...",
      "content": "Persimmon Monaka 5 pieces​​ Enjoy the invigorating aroma of persimmon and yuzu, created using fruits sourced from Nara and Yoshino. This",
      "score": 0.8252664,
      "raw_content": null
    }
  ],
  "response_time": 0.6,
  "request_id": "45367771-21da-41d

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Chain: Tavily -> format results
context_chain = tavily_search | format_tavily_results

prompt_text = """
Answer the user's question based only on the "Knowledge" below.
Do not guess information that is not written in the Knowledge; instead, respond with "I don't know from the provided knowledge."
If possible, append the source numbers you referenced at the end of your answer (e.g., [1][2]).
Finally, list the source numbers and URLs as bullet points.

# Knowledge:
{context}
"""

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    ("human", "Question: {question}"),
])

# Create the model
model = ChatOpenAI(model=MODEL_NAME)

my_chain = (
    {"context": context_chain,
     "question": RunnablePassthrough()}
    | prompt | model | str_output_parser
)

# Run the chain
response = my_chain.invoke("What are some well-known souvenirs from Nara Prefecture?")

# Display the result
print(response)


Well-known souvenirs from Nara Prefecture include:

- Nara Deer Cookies (Shika Senbei) [1]  
- Nara uchiwa (traditional fans) [1]  
- Yamato Cha (Nara green tea) [1]  
- Kakinoha sushi (kakinoha-zushi) [1]  
- Persimmon monaka (persimmon-and-yuzu sweet) [2]  

[1][2]

Sources:
- [1] https://spejapa-excursion.com/blog/b-70.html  
- [2] https://livejapan.com/en/in-kansai/in-pref-nara/in-nara_ikoma_tenri/article-a2000428/


## 7. Conditional Branching

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.router import RouterRunnable
from pydantic import BaseModel, Field  # For defining structured data

# Class definition for decision logic
class SearchDecision(BaseModel):
    """A model for determining whether a web search is needed."""
    needs_search: bool = Field(
        description="True if a web search is required, False if it is not."
    )


In [11]:
# A chain to determine whether a web search is needed
needs_search_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}\n\nDo you need to perform a web search to answer this question?")
])

needs_search_chain = (
    needs_search_prompt
    | model.with_structured_output(SearchDecision)
    | (lambda x: "search" if x.needs_search else "no_search")  # Convert the decision into a routing key
)


In [12]:
# Chain that performs the search
context_chain = tavily_search | format_tavily_results

prompt_text = """
Answer the user's question based only on the "Knowledge" below.
Do not guess information that is not written in the Knowledge; instead, respond with "I don't know from the provided knowledge."
If possible, append the source numbers you referenced at the end of your answer (e.g., [1][2]).
Finally, list the source numbers and URLs as bullet points.

# Knowledge:
{context}
"""

search_answer_prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    ("human", "Question: {question}"),
])

search_chain = (
    {"context": context_chain,
     "question": RunnablePassthrough()}
    | search_answer_prompt | model | str_output_parser
)


In [13]:
# Chain that answers without performing a search
no_search_answer_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}\n\nPlease answer the question above.")
])

no_search_chain = (
    no_search_answer_prompt
    | model
    | StrOutputParser()
)


In [14]:
# Branching with RouterRunnable
# Execute the Runnable corresponding to the given key
router = RouterRunnable(
    runnables={
        "search": search_chain,
        "no_search": no_search_chain,
    }
)

# RouterRunnable expects input in the form {"key": "...", "input": ...}
final_chain = (
    {"key": needs_search_chain, "input": RunnablePassthrough()}
    | router
)


In [15]:
print("--- Run: What is the price of the iPhone 16e? ---")
# needs_search_chain returns "search" -> search_chain is executed
print(final_chain.invoke("What is the price of the iPhone 16e?"))

print("\n--- Run: What is 1+1? ---")
# needs_search_chain returns "no_search" -> no_search_chain is executed
print(final_chain.invoke("What is 1+1?"))


--- Run: What is the price of the iPhone 16e? ---
The iPhone 16e starts at $599 USD. [1][2]

Sources:
- [1] https://prices.appleinsider.com/iphone-16e
- [2] https://www.facebook.com/cnet/videos/iphone-16e-review/4161070920885616/

--- Run: What is 1+1? ---
1 + 1 = 2.
